In [1]:
import scanpy as sc
import pandas as pd
import os
from dca.api import dca
import tensorflow as tf
from tensorflow.python.keras.backend import set_session

2021-08-03 18:04:07.267438: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-03 18:04:07.267456: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/kopt/config.py:60: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  _config = yaml.load(open(_config_path))


### INPUT: Specify which sample you want to analyze. Currently available are:

- Sample2_70_percent
- Sample2_80_percent
- Sample3_default
- Sample4_70_percent
- Sample4_80_percent
- Sample5_70_percent
- Sample5_80_percent

In [2]:
sample = 'Sample5_80_percent'

### Loading the read count data and potentially removing cell doublets/mulitplets.

In [3]:
dat = pd.read_table('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/tsv/' + sample + '.barcode.cell.distribution.tsv',sep='\t')
remove_mixed = True
if remove_mixed:
    clust_file = pd.read_csv('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/'+ sample + '/tsv/cluster_assignment.tsv',
                            sep='\t')
    drop_rows = clust_file['Barcode'][clust_file['CellType']=='Mixed']
    dat = dat.drop(drop_rows)

### Creating an AnnData object from the read counts.

In [4]:
adata = sc.AnnData(dat)
sc.pp.filter_genes(adata,min_cells=1)

### Running the autoencoder: Currently, we have the following available architectures:

- 'meth-encoder': A classical autoencoder, where the dispersion parameters of both NB-distributions can be freely selected by the model.
- 'meth-encoder-constant': An autoencoder, where the dispersion parameters is fixed for each gene. This lowers the number of paramters substantially.
- 'meth-encoder-poisson': An autoencoder, which mixes a negative bionomial distribution (foreground) with a Poisson distribution as the background
- 'meth-encoder-poisson-constant': Same as the above, but with a fixed dispersion for each gene
- 'meth-encoder-simple': Instead of having a dense layer representing the mean per cell and amplicon, we use a constant mean per amplicon for both the foreground and the background distribution.

In [5]:
res = dca(adata,
          ae_type='meth-simple-encoder',
          return_info=True,
          return_model=True,
          return_bottleneck=True,
          verbose=True,
          early_stop=50,
          epochs=3000,
          network_kwds={'debug': True},
          init='glorot_normal',
          hidden_size=(16,8,16))

2021-08-03 18:04:09.316161: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-03 18:04:09.316302: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-03 18:04:09.316314: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-03 18:04:09.316330: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CZC041BTPK): /proc/driver/nvidia/version does not exist


dca: Successfully preprocessed 199 genes and 5207 cells.



2021-08-03 18:04:09,483 [WARNING] From /users/lvelten/project/Methylome/src/error_mod/dca/dca/train.py:41: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

2021-08-03 18:04:09.485185: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              [(None, 199)]        0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 16)           3200        count[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16)           48          enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 16)           0           batch_normalization[0][0]        
____________________________________________________________________________________________

2021-08-03 18:04:10.231896: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2021-08-03 18:04:10.248573: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


...

4686/4686 [==============================] - 0s 42us/sample - loss: 6.3974 - val_loss: 6.4256
Epoch 2680/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3973 - val_loss: 6.4259
Epoch 2681/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3972 - val_loss: 6.4251
Epoch 2682/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3971 - val_loss: 6.4280
Epoch 2683/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3970 - val_loss: 6.4272
Epoch 2684/3000
4686/4686 [==============================] - 0s 43us/sample - loss: 6.3969 - val_loss: 6.4264
Epoch 2685/3000
4686/4686 [==============================] - 0s 45us/sample - loss: 6.3969 - val_loss: 6.4249
Epoch 2686/3000
4686/4686 [==============================] - 0s 45us/sample - loss: 6.3968 - val_loss: 6.4249
Epoch 2687/3000
4686/4686 [==============================] - 0s 45us/sample - loss: 6.3967 - val_loss: 6.4255
Epoch 2688/3000
4686/4686 

4686/4686 [==============================] - 0s 41us/sample - loss: 6.3914 - val_loss: 6.4191
Epoch 2754/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3913 - val_loss: 6.4192
Epoch 2755/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3913 - val_loss: 6.4191
Epoch 2756/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3911 - val_loss: 6.4194
Epoch 2757/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3911 - val_loss: 6.4195
Epoch 2758/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3911 - val_loss: 6.4192
Epoch 2759/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3909 - val_loss: 6.4194
Epoch 2760/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3910 - val_loss: 6.4188
Epoch 2761/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3908 - val_loss: 6.4198
Epoch 2762/3000
4686/4686 

4686/4686 [==============================] - 0s 41us/sample - loss: 6.3857 - val_loss: 6.4145
Epoch 2828/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3857 - val_loss: 6.4138
Epoch 2829/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3857 - val_loss: 6.4179
Epoch 2830/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3855 - val_loss: 6.4131
Epoch 2831/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3854 - val_loss: 6.4148
Epoch 2832/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3854 - val_loss: 6.4129
Epoch 2833/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3853 - val_loss: 6.4149
Epoch 2834/3000
4686/4686 [==============================] - 0s 40us/sample - loss: 6.3852 - val_loss: 6.4155
Epoch 2835/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3851 - val_loss: 6.4144
Epoch 2836/3000
4686/4686 

4686/4686 [==============================] - 0s 41us/sample - loss: 6.3804 - val_loss: 6.4081
Epoch 2902/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3803 - val_loss: 6.4082
Epoch 2903/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3803 - val_loss: 6.4082
Epoch 2904/3000
4686/4686 [==============================] - 0s 42us/sample - loss: 6.3802 - val_loss: 6.4086
Epoch 2905/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3802 - val_loss: 6.4095
Epoch 2906/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3800 - val_loss: 6.4078
Epoch 2907/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3799 - val_loss: 6.4074
Epoch 2908/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3799 - val_loss: 6.4074
Epoch 2909/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3799 - val_loss: 6.4073
Epoch 2910/3000
4686/4686 

4686/4686 [==============================] - 0s 40us/sample - loss: 6.3753 - val_loss: 6.4035
Epoch 2976/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3752 - val_loss: 6.4033
Epoch 2977/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3752 - val_loss: 6.4031
Epoch 2978/3000
4686/4686 [==============================] - 0s 42us/sample - loss: 6.3751 - val_loss: 6.4026
Epoch 2979/3000
4686/4686 [==============================] - 0s 40us/sample - loss: 6.3750 - val_loss: 6.4027
Epoch 2980/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3749 - val_loss: 6.4024
Epoch 2981/3000
4686/4686 [==============================] - 0s 40us/sample - loss: 6.3749 - val_loss: 6.4029
Epoch 2982/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3748 - val_loss: 6.4021
Epoch 2983/3000
4686/4686 [==============================] - 0s 41us/sample - loss: 6.3747 - val_loss: 6.4042
Epoch 2984/3000
4686/4686 

### Writing the data on disk.

In [6]:
di = '/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/methylation_autoencoder/'
if not os.path.isdir(di):
    os.mkdir(di)
    
pd.DataFrame(adata.obsm['X_meth_value']).to_csv(di + 'mixture_prob_dca_test.csv')

In [7]:
foo = res.get_encoder().predict([adata.X, adata.X.mean(1)])

In [8]:
pd.DataFrame(adata.obsm['X_bottleneck']).to_csv(di + 'bottleneck.csv')

In [9]:
adata.obsm['mean1_norm']

array([[159.77126],
       [139.99767],
       [167.62038],
       ...,
       [188.73166],
       [137.76099],
       [148.5073 ]], dtype=float32)

In [10]:
adata.obsm['mean2_norm']

array([[ 63.69657 ],
       [ 39.869816],
       [ 71.896454],
       ...,
       [100.23306 ],
       [ 37.72623 ],
       [ 47.666855]], dtype=float32)

In [11]:
adata.obsm['alpha']

KeyError: 'alpha'